In [ ]:
cd examples/text_to_image/

In [ ]:
pip install git+https://github.com/huggingface/diffusers.git

In [ ]:
pip install -U -r requirements.txt


In [ ]:
!accelerate config default

In [ ]:
#This works 

import os
models_list = [
    "stabilityai/stable-diffusion-xl-base-1.0",
    "CompVis/stable-diffusion-v1-4",
    "stablediffusionapi/anything-v5",
    "waifu-diffusion v1.4"


]

# Set environment variables
os.environ['MODEL_NAME'] = "CompVis/stable-diffusion-v1-4"
os.environ['DATASET_NAME'] = "rod101/Anime1K"


!accelerate launch train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --mixed_precision="fp16" \
  --max_train_steps=200 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="sdxl-anime-model"


In [ ]:
# INFERENCE

import torch
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt
from PIL import Image

# Load the fine-tuned model
model_path = "sdxl-anime-model"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = None
# Run inference
prompt = "a man with dark hair and brown eyes mask"
num_images = 1

with torch.autocast("cuda"):
    images = pipe(prompt, num_images_per_prompt=num_images).images

# Display the generated images


for img in images:
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()
